password: 4n=KTc8mxoKwp=UCKfw_
token: eyJ2ZXIiOiI4LjYuMiIsImFkciI6WyIxOTIuMTY4LjAuMTExOjkyMDAiXSwiZmdyIjoiYzE0OWY2ZDllOWU4MjQ3ZjE0MzdmMDRlMjE3MDBkNmQzNWNmYThjYTk2ZmM5OTY1YTBjYjZlMjNmNjgxYTJhNCIsImtleSI6InY4Q3Fqb2dCbDBFRzZNRndBQS1qOlpTUFczaGpKUU0tdlVyWndYZFNFU2cifQ==

In [ ]:
%pip install elasticsearch

## Kết nối Elastic

In [ ]:
from elasticsearch import Elasticsearch
import os

HOST = "https://localhost:9200"
ELASTIC_USER = "elastic"
# The password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = "4n=KTc8mxoKwp=UCKfw_"
# The path of ca certificates 
CA_CERTS = os.path.realpath(os.path.join(os.path.dirname(__name__), 'elasticsearch-8.6.2', 'config', 'certs', 'http_ca.crt'))

# Create the client instance
es = Elasticsearch(
    hosts = HOST,
    ca_certs = CA_CERTS,
    http_auth = (ELASTIC_USER, ELASTIC_PASSWORD),
)
index_name = "product"
type_name = "_doc"
path = os.path.realpath(os.path.join(os.path.dirname(__name__), 'documents'))
es.info()

### Tạo index

In [ ]:
import json

if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
else:
    print("index does not exists")
es.indices.create(index=index_name)

### Định nghĩa dữ liệu

In [ ]:
body = {
    "properties": {
        "asin": {"type": "keyword"},
        "brand": {
            "type": "text",
            "term_vector": "with_positions_offsets"
        },
        "stars": {"type": "text"},
        "reviewsCount": {"type": "text"},
        "thumbnailImage": {"type": "text"},
        "description": {"type": "text"},
        "url": {"type":"text"},
        "price": {
            "properties":{
                "value": {"type": "text"},
                "currency": {"type": "text"},
            }
        },
        "title": {
            "type": "text",
            "term_vector": "with_positions_offsets"
        },
        "breadCrumbs": {"type":"text","term_vector": "with_positions_offsets"},
    }
}    
es.indices.put_mapping(index = index_name, body=body)

### Load dữ liệu vào index

In [ ]:
import os
i = 0

for filename in os.listdir(path):
    if filename.endswith('.json'):
        with open(os.path.join(path, filename)) as open_file:
            content = open_file.read()
            data = json.loads(content)
            try:
                data['price']['value'] = float(data['price']['value'])
            except Exception as ex:
                data['price']['value'] = 5.9
            es.index(index=index_name, ignore=400, id=i, body=data)
            i = i + 1

### Kiểm tra

In [ ]:
es.get(index=index_name, id=1)["_source"]

In [ ]:
mapping = es.indices.get_mapping(index=index_name)
print(mapping)

## Cài đặt phân tích

In [ ]:
body = {
    "settings": {
        "analysis": {
            "analyzer": {
                "my_analyzer": {
                    "tokenizer": "standard",
                    "filter": ["lowercase"]
                }
            },
            "tokenizer": {
                "my_tokenizer": {
                    "type": "pattern",
                    "pattern": "\"((?:\\\\\"|[^\"]|\\\\\")+)\" ",
                    "group": 1
                }
            }
    }
  }
}
es.indices.close(index=index_name)
es.indices.put_settings(index=index_name, body=body)
es.indices.open(index=index_name)

## Tạo hàm Ranking

In [ ]:
body = {
    "settings": {
        "index": {
            "similarity": {
                "my_similarity": {
                    "type": "DFR",
                    "basic_model": "g",
                    "after_effect": "l",
                    "normalization": "h2",
                    "normalization.h2.c": "3.0"
                }
            }
        }
    }
}
es.indices.close(index=index_name)
es.indices.put_settings(index=index_name, body=body)
es.indices.open(index=index_name)

## Kiểm tra searching

In [ ]:
body = {
    "query": {
        "multi_match": {
            "query": "Glue", 
            "fields": ["brand", "title"], 
            "operator": "and"
        }
    }
}
products = [p['_source'] for p in es.search(index=index_name, body=body)["hits"]["hits"]]
print(products)